In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as pyo
import numpy as np
import os
from tqdm.notebook import tqdm
import cufflinks as cf               # third-party library for Pandas 
import plotly.graph_objs as go
%matplotlib inline
cf.go_offline()
pyo.init_notebook_mode(connected=True)
import ipywidgets as widgets
import itertools

In [114]:
file_lenght=-len(os.path.abspath(".").split("\\")[-1])
csv_path =  os.path.abspath(".")[:file_lenght]+"Data\\"

types={       
    "PARTNERCATEGORY_CODE" : "category" 
    ,"PARTNERCATEGORYTYPE_CODE" : "category" 
    ,"VENDOR" : "category"           
    ,"SECTOR CODE" : "category"         
    ,"SECTOR NAME" : "category"       
    ,"REGION CODE" : "category"        
    ,"REGION NAME" : "category"       
    ,"PRODUCT_CODE" : "category"      
    ,"PRODUCT_NAME" : "category"       
    ,"PRODUCTPAGE_CODE" : "category"    
    ,"UNITPRICE" : "float16"
    ,"SOLD QUANTITY" : "float16"  
    ,"TURNOVER" : "float64"
    ,"DISCOUNT" : "float32" 
    ,"HOLIDAY" : "category"    
    ,"SCOLAR_HOLIDAY" : "category"     
    ,"ISLAM_EVENT" : "category"    
    ,"TEMPERATURE" : "float16"
    ,"HUMIDITY" : "float16"
} # reduce memory consumption
miniData = pd.read_csv(csv_path+"Boufarik_Dataset_v2.csv",header=0,engine='c',index_col=[0],dtype=types)
miniData= miniData.reset_index() 

In [116]:
month ={'01':'Janauary',
        '02':'February',
        '03':'March',
        '04':'April',
        '05':'May',
        '06':'June',
        '07':'July',
        '08':'August',
        '09':'September',
        '10':'October',
        '11':'November',
        '12':'December'}
flipped_month = dict(zip(month.values(), month.keys()))

def f_year(date):
    return date[:4]
def f_month(date):
    return month[date[5:7]]
vect_month = np.vectorize(f_month)
vect_year = np.vectorize(f_year)

month_col = vect_month(miniData["GREGORIAN_DATE"])
month_cat = pd.CategoricalDtype(categories=flipped_month,ordered=True)

year_col = vect_year(miniData["GREGORIAN_DATE"])
year_cat = pd.CategoricalDtype(categories=["2016","2017","2018"],ordered=True)

miniData["MONTH"]=pd.Series(month_col).astype(month_cat)
miniData["YEAR"]=pd.Series(year_col).astype(year_cat)

In [117]:
gammes = miniData["PRODUCTPAGE_CODE"].drop_duplicates().tolist()
data1= list()
data = []
for gamme in tqdm(gammes):
    produits = miniData[miniData["PRODUCTPAGE_CODE"].str.match(gamme)]["PRODUCT_NAME"].drop_duplicates().tolist()
    for prod in produits:
        data1.append(miniData[miniData["PRODUCT_NAME"].str.match(prod)].drop_duplicates(["MONTH","YEAR"])[["PRODUCTPAGE_CODE","PRODUCT_NAME","MONTH","YEAR"]].iloc[0,:])
    data.append(pd.concat(data1, axis=1).transpose().set_index(["PRODUCTPAGE_CODE","PRODUCT_NAME"]))
    data1=list()
pd.concat(data)

MONTH  YEAR
PRODUCTPAGE_CODE    PRODUCT_NAME                          
laits aromatisÃ©s   CHOCO 20CL              Janauary  2016
                    CHOCO 1L                Janauary  2016
                    FRAISE 20CL            September  2016
                    BANANE 125 ML            October  2016
                    CHOCO 125 ML             October  2016
                    FRAISE 125 ML           November  2016
                    CANDY CARAMEL 125 ML   September  2017
Laits Blancs        ENTIER                  Janauary  2016
                    DE 1L                   Janauary  2016
                    SILHOUETTE              Janauary  2016
                    VIVA                    Janauary  2016
                    DE 50CL                 Janauary  2016
                    DE 1L SANS LACTOSE        August  2018
Boissons            COCKTAIL 1L             Janauary  2016
                    COCKTAIL 20CL           Janauary  2016
                    ORANGE 1L               Janauary  2016
                    ORANGE 20CL             Janauary  2016
                    CITRONNADE                 March  2016
                    NECTAR DE GRENADE        October  2016
Laits & Jus         TWIST O-F-B 20CL        Janauary  2016
                    TWIST O-M 20CL          Janauary  2016
                    TWIST P-A 20CL          Janauary  2016
                    TWIST O-A 20CL          Janauary  2016
                    TWIST O-A 1L            Janauary  2017
                    TWIST P-A 1L            February  2017
                    TWIST O-M 1L            February  2017
                    TWIST O-F-B 1L             March  2017
                    TWIST O-F-B 125ML      September  2018
                    TWIST O-A 125ML        September  2018
Le Maitre Cuisinier PREPARATION CULINAIRE        May  2017
POUDRE ETUI         ETUI 500G               December  2018
Le Maitre Glacier   ICE CREAM - CHOCOLAT        July  2018

## Monthly behaviour in BOUFARIK

In [41]:
month_seas=miniData.groupby(["MONTH","PARTNERCATEGORY_CODE"]).sum().dropna()["TURNOVER"].reset_index()
classes = miniData["PARTNERCATEGORY_CODE"].drop_duplicates().tolist()
data = [month_seas[month_seas["PARTNERCATEGORY_CODE"].str.match(classe)].groupby("MONTH").sum() for classe in classes]

fig = go.Figure(data=[ go.Bar(name=classes[i], x=data[i].index, y=data[i]["TURNOVER"].tolist()) for i in range(len(classes)) ])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [71]:
monthyear_seas=miniData.groupby(["YEAR","MONTH","PRODUCTPAGE_CODE"]).sum().dropna()["TURNOVER"].reset_index()
gammes = miniData["PRODUCTPAGE_CODE"].drop_duplicates().tolist()
data = [
        monthyear_seas[monthyear_seas["PRODUCTPAGE_CODE"].str.match(gamme)].groupby(["MONTH","YEAR"]).sum().groupby("MONTH").mean() 
    for gamme in gammes ]

fig = go.Figure(data=[ go.Bar(name=gammes[i], x=data[i].index, y=data[i]["TURNOVER"].tolist()) for i in range(len(gammes)) ])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [72]:
monthyear_seas=miniData[miniData["PRODUCTPAGE_CODE"].str.match("Laits Blancs")].groupby(["YEAR","MONTH","PRODUCT_NAME"]).sum().dropna()["TURNOVER"].reset_index()
gammes = miniData[miniData["PRODUCTPAGE_CODE"].str.match("Laits Blancs")]["PRODUCT_NAME"].drop_duplicates().tolist()
data = [
        monthyear_seas[monthyear_seas["PRODUCT_NAME"].str.match(gamme)].groupby(["MONTH","YEAR"]).sum().groupby("MONTH").mean() 
    for gamme in gammes ]

fig = go.Figure(data=[ go.Bar(name=gammes[i], x=data[i].index, y=data[i]["TURNOVER"].tolist()) for i in range(len(gammes)) ])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [74]:
monthyear_seas=miniData[miniData["PRODUCTPAGE_CODE"].str.contains("laits aro")].groupby(["YEAR","MONTH","PRODUCT_NAME"]).sum().dropna()["TURNOVER"].reset_index()
gammes = miniData[miniData["PRODUCTPAGE_CODE"].str.contains("laits aro")]["PRODUCT_NAME"].drop_duplicates().tolist()
data = [
        monthyear_seas[monthyear_seas["PRODUCT_NAME"].str.match(gamme)].groupby(["MONTH","YEAR"]).sum().groupby("MONTH").mean() 
    for gamme in gammes ]

fig = go.Figure(data=[ go.Bar(name=gammes[i], x=data[i].index, y=data[i]["TURNOVER"].tolist()) for i in range(len(gammes)) ])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

## Weekly behaviour in BOUFARIK

In [124]:
WeekDays= {0:'Monday',
            1:'Tuesday',
            2:'Wednesday',
            3:'Thursday',
            4:'Friday',
            5:'Saturday',
            6:'Sunday'}
flipped_week = dict(zip(WeekDays.values(), WeekDays.keys()))

def f_day(date):
    return WeekDays[date]

vect_day = np.vectorize(f_day)


day_col = vect_day(miniData["DAYWEEK"])
day_cat = pd.CategoricalDtype(categories=flipped_week,ordered=True)

miniData["DAYWEEK"]=pd.Series(day_col).astype(day_cat)

In [128]:
month_seas=miniData.groupby(["DAYWEEK","PARTNERCATEGORY_CODE"]).sum().dropna()["TURNOVER"].reset_index()
classes = miniData["PARTNERCATEGORY_CODE"].drop_duplicates().tolist()
data = [month_seas[month_seas["PARTNERCATEGORY_CODE"].str.match(classe)].groupby("DAYWEEK").mean() for classe in classes]

fig = go.Figure(data=[ go.Bar(name=classes[i], x=data[i].index, y=data[i]["TURNOVER"].tolist()) for i in range(len(classes)) ])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [133]:
week_seas=miniData.groupby(["DAYWEEK","PRODUCTPAGE_CODE"]).sum().dropna()["TURNOVER"].reset_index()
gammes = miniData["PRODUCTPAGE_CODE"].drop_duplicates().tolist()
data = [
        week_seas[week_seas["PRODUCTPAGE_CODE"].str.match(gamme)].groupby("DAYWEEK").mean() 
    for gamme in gammes ]

fig = go.Figure(data=[ go.Bar(name=gammes[i], x=data[i].index, y=data[i]["TURNOVER"].tolist()) for i in range(len(gammes)) ])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [139]:
month_seas=miniData.groupby(["DAYWEEK","PARTNERCATEGORY_CODE"]).count().dropna()["TURNOVER"].reset_index()
classes = miniData["PARTNERCATEGORY_CODE"].drop_duplicates().tolist()
data = [month_seas[month_seas["PARTNERCATEGORY_CODE"].str.match(classe)].groupby("DAYWEEK").mean() for classe in classes]

fig = go.Figure(data=[ go.Bar(name=classes[i], x=data[i].index, y=data[i]["TURNOVER"].tolist()) for i in range(len(classes)) ])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [140]:
miniData[miniData["PARTNERCATEGORY_CODE"].str.match("M")].count()

index                       78164
ITINERARY_NAME              78164
PARTNER_CODE                78164
PARTNERCATEGORY_CODE        78164
PARTNERCATEGORYTYPE_CODE    78164
VENDOR                      78164
SECTOR CODE                 78164
SECTOR NAME                 78164
REGION CODE                 78164
REGION NAME                 78164
PRODUCT_CODE                78164
PRODUCT_NAME                78164
PRODUCTPAGE_CODE            78164
UNITPRICE                   78164
SOLD QUANTITY               78164
TURNOVER                    78164
DISCOUNT                    78164
GREGORIAN_DATE              78164
HOLIDAY                     78164
SCOLAR_HOLIDAY              78164
ISLAM_EVENT                 78164
TEMPERATURE                 77843
HUMIDITY                    77807
DAYWEEK                     78164
LOYALTY                     78164
MONTH                       78164
YEAR                        78164
dtype: int64